In [1]:
from mmengine.config import Config
from mmengine.registry import RUNNERS
from mmengine.runner import Runner

cfg = Config.fromfile("softmoe/configs/timm/vit_tiny_moe_timm.py")
cfg.work_dir = './work_dirs/test'

if 'runner_type' not in cfg:
    # build the default runner
    runner = Runner.from_cfg(cfg)
else:
     # build customized runner from the registry
    # if 'runner_type' is set in the cfg
    runner = RUNNERS.build(cfg)


03/25 22:02:17 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.8.18 (default, Sep 11 2023, 13:40:15) [GCC 11.2.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 3407
    GPU 0,1,2,3,4,5,6,7: NVIDIA A800 80GB PCIe
    CUDA_HOME: /usr/local/cuda-12.1
    NVCC: Cuda compilation tools, release 12.1, V12.1.66
    GCC: gcc (GCC) 4.8.5 20150623 (Red Hat 4.8.5-44)
    PyTorch: 2.1.0
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2023.1-Product Build 20230303 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.1.1 (Git Hash 64f6bcbcbab628e96f33a62c3e975f8535a7bde4)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX512
  - CUDA Runtime 12.1
  - NVCC architecture flags: -gencode;arch=compute_50

In [2]:
data_loader_val = runner.val_dataloader
model = runner.model

In [3]:
import torch
import time


@torch.no_grad()
def throughput(data_loader, model):
    model.eval()

    for data in data_loader:
        images = torch.stack(data['inputs']).cuda()
        batch_size = images.shape[0]
        proced_data = model.data_preprocessor(data)
        for i in range(50):
            model(proced_data['inputs'])
        torch.cuda.synchronize()
        print(f"throughput averaged with 30 times")
        tic1 = time.time()
        for i in range(30):
            model(proced_data['inputs'])
        torch.cuda.synchronize()
        tic2 = time.time()
        print(f"batch_size {batch_size} throughput {30 * batch_size / (tic2 - tic1)}")
        return

In [5]:
throughput(data_loader_val, model)  # batch_size 64 throughput 2044.938957036053 197*4moe:1572.249532946206 128*4moe:1723.6773518977318

throughput averaged with 30 times
batch_size 64 throughput 1538.4231526173237
